In [1]:
!ls -l
import json

total 2408
-rw-r-----@ 1 yoavfreund  503     77444 Sep 22 16:28 CSE103_Fall14_Week2_problem13_part1.txt
-rw-r-----@ 1 yoavfreund  503    196948 Sep 22 16:28 CSE103_Fall14_Week2_problem13_part2.txt
-rw-r-----@ 1 yoavfreund  503    246614 Sep 22 16:29 CSE103_Fall14_Week2_problem13_part3.txt
-rw-r-----@ 1 yoavfreund  503    282073 Sep 22 16:29 CSE103_Fall14_Week2_problem13_part4.txt
-rw-r-----@ 1 yoavfreund  503    177009 Sep 22 16:30 CSE103_Fall14_Week2_problem13_part5.txt
-rw-r--r--  1 yoavfreund  staff   33019 Sep 22 09:27 Function pointers and multiprocess communication.ipynb
-rw-r--r--  1 yoavfreund  staff    5761 Sep 22 09:27 Load Libraries.ipynb
-rw-r--r--  1 yoavfreund  staff    2878 Sep 17 12:57 Problem2.3.json
-rw-r--r--  1 yoavfreund  staff   65469 Sep 17 12:57 Problem8Attempts.json
-rw-r--r--  1 yoavfreund  staff   46568 Sep 22 09:27 Process Data.ipynb
-rw-r--r--  1 yoavfreund  staff   23921 Sep 22 09:27 Week3.Prob8.Part1.json
-rw-r--r--  1 yoavfreund  staff   36459 Sep 17 12:

In [182]:

%cd ../filter_helpers/

/Users/yoavfreund/projects/Webwork.Improvement.Project/AdaptiveHints/src/servers/rest_server/filter_helpers


In [183]:
%%writefile classify_final_value.py
def classify_final_value(attempt_flat,answer_flat):
    """ Written by Yoav Freund, Sat Sep 19 17:09:48 PDT 2015
    classifies the final value as: correct,parse_error,int, not_int
    """
    
    att_final_value=attempt_flat[0]['val']
    ans_final_value=answer_flat[0]['val']
    
    if att_final_value==ans_final_value:
        return 'correct'
    if isinf(att_final_value):
        return "inf"
    if int(att_final_value)!=att_final_value:
        return "not int"
    else:
        return "int"

Writing classify_final_value.py


In [184]:
%%writefile answer_is_not_expression.py        
def answer_is_not_expression(answer_string, parse_tree, eval_tree, correct_string, correct_tree, correct_eval, user_vars):
    if len(att_eval_tree)==1:
        return "Please enter an expression, not just final numerical result"
    else:
        return ""

Writing answer_is_not_expression.py


In [97]:
def check_81(attempt,att_parse_tree,att_eval_tree,answer,answer_parse_tree,answer_eval_tree,variables):

    attempt_class=classify_final_value(att_eval_tree,answer_eval_tree)
    if  attempt_class=='correct':
        return 'correct answer'
    if attempt_class=='int':
        return 'Can the answer be an integer number? [_____]{n}'
    
    hint=answer_is_not_expression(attempt,att_parse_tree,att_eval_tree,answer,answer_parse_tree,answer_eval_tree,variables)
    if  len(hint)!=0: 
        return hint

    return None

In [17]:
cd ../filter_helpers/

/Users/yoavfreund/projects/Webwork.Improvement.Project/AdaptiveHints/src/servers/rest_server/filter_helpers


In [185]:
%%writefile flatten.py
from numbers import Number
def flatten(L):
    if type(L)!=list:
        #print 'not list'
        return None
    elif len(L)==1 and isinstance(L[0],Number):
        #print 'list of one number'
        return [{'val':L[0]}]
    elif len(L)==3 and type(L[0])==unicode:
        if not isinstance(L[1],Number):
            print "expected number, but got %s, in %s"%(str(L[1]),str(L))
            return None
        if type(L[2])==list:
            if len(L[2])==2:
                if type(L[2][0])==int and type(L[2][1])==int:
                    return [{'val':L[1],'location':L[2]}]
        print "expected a tuple of the form (operation,value,[start,end])"
        print "instead got "+str(L)
        return None
    else:
        #print 'general list'
        answer=[]
        for item in L:
            fitem=flatten(item)
            if fitem==None:
                return None
            answer = answer+fitem
        return answer

Writing flatten.py


In [186]:
%%writefile find_common.py
def find_range(item,flat_tree):
    for entry in flat_tree:
        if item==entry['val']:
            return entry['location']
    return None
    
def find_common(attempt,att_flat,answer,ans_flat):
    "Find sub-expressions common to the attempt and the answer"

    att_set=[x['val'] for x in att_flat if x.has_key('val')]
    answer_set=[x['val'] for x in ans_flat if x.has_key('val')]

    intersection=[x for x in set(att_set) & set(answer_set) if (int(x)!=x) or x>100]
    
    if len(intersection)==0:
        return None
 
    print 'common subexpressions=',str(intersection)
    #print 'attempt=%s, att_eval_tree=%s\n variables=%s,answer=%s,answer_eval_tree=%s'\
    #    %(attempt,str(att_eval_tree),str(variables),answer,str(answer_eval_tree))

    for item in intersection:
        att_range=find_range(item,att_flat)
        ans_range=find_range(item,ans_flat)
        print "attempt subexpression %s is equivalent to answer subexpression %s"\
        %(attempt[att_range[0]:att_range[1]+2],answer[ans_range[0]:ans_range[1]+2])
 
        

Writing find_common.py


In [187]:
%%writefile substit.py
from string import replace
def substit(string,variables):
    for var in variables.keys():
        val=str(variables[var])
        string=string.replace(var,val)
    return string

Writing substit.py


In [195]:
#%%writefile main.py
file=open('CSE103_Fall14_Week2_problem13_part5.txt','r');
ID=file.readline()
answer=file.readline()
print 'answer=',answer
count =0
for line in file.readlines():
    count+=1
    if count>10: break
    params=json.loads(line);
    attempt,att_parse_tree,att_eval_tree,answer,answer_parse_tree,answer_eval_tree,variables = params
    #print 'answer before substitution',answer
    answer=substit(answer,variables)
    #print 'answer after substitution',answer
    attempt_flat=flatten(att_eval_tree)
    answer_flat =flatten(answer_eval_tree)
    #print attempt_flat
    #print answer_flat
    if attempt_flat==None or answer_flat==None:
        print 'Eval tree== None or parsing error ------------------------------'
        print str(params)
        break

    Class=classify_final_value(attempt_flat,answer_flat)
    if Class=='correct' or Class=='int':
        print Class
        continue

    #print "attempt={:<40} answer={:<40}".format(attempt,answer)
    hint= find_common(attempt,attempt_flat,answer,answer_flat)
    

answer= ($coins!/(6!($coins-6)!))/2^$coins

Eval tree== None or parsing error ------------------------------
[u'462/2^10', [[u'/', [0, 6]], 462, [[u'^', [4, 6]], 2, 10]], [[u'/', 0.451171875, [0, 6]], [462.0], [[u'^', 1024.0, [4, 6]], [2.0], [10.0]]], u'', None, 0, {u'$coins': 11.0}]


In [153]:
"{:<7}{:<51}{:<25}\n".format('this', 'that', 'and the other')

'this   that                                               and the other            \n'

In [193]:
%cd filter_lab

/Users/yoavfreund/projects/Webwork.Improvement.Project/AdaptiveHints/src/servers/rest_server/filter_lab


In [9]:
defined X

SyntaxError: invalid syntax (<ipython-input-9-56d3901bedac>, line 1)

In [43]:
from numpy import isinf,isn
isinf(float('inf'))

True

In [83]:
import numbers

In [88]:
isinstance('10',numbers.Number)

False

In [92]:
'abs'=='abd'

False

In [106]:
answer

u''

In [107]:
X=None
X==None

True